# Inference

In [10]:
# Ensure local package is importable in this notebook (editable install) and add project root to sys.path.
from pathlib import Path
import sys

project_root = str(Path("..").resolve())
if project_root not in sys.path:
	sys.path.insert(0, project_root)

from src.models.ebm_predictor import EBMPredictor

In [11]:
# FPKM coverage files path
base_path = '/mnt/d/flync_final_runs/Vu_GSE199164_3d_vs_7d/vu_gse199164/cov/'

In [12]:
predictor = EBMPredictor(
    model_path="/home/chlab/flync/final_train_artifacts/FINAL_flync_ebm_model.pkl",
    schema_path="/home/chlab/flync/final_train_artifacts/flync_model_schema.json",
    check_value_ranges=True
)

In [13]:
import pandas as pd

inference = pd.read_parquet("cleaned_dataset_3d_7d.parquet")
res = pd.read_parquet('features.parquet').set_index('index')

inference.shape, res.shape

((24219, 731), (27021, 43))

In [14]:
res_cols = ['transcript_id', 'chromosome', 'start', 'end', 'ss_sequence', 'ss_structure']

res = res[[c for c in res.columns if c in res_cols]]

In [15]:
res

,chromosome,start,end,ss_sequence,ss_structure
index,,,,,
MSTRG.2.1,chr2L,9837,21424,AAAUAUACCUUCUAAAAAAUAUAUGAUUUUUUCGAUAGAGAUACAC...,.((((((.((((((((((((.....))))))).(((((((((((.....
MSTRG.2.2,chr2L,9838,21424,AAAUAUACCUUCUAAAAAAUAUAUGAUUUUUUCGAUAGAGAUACAC...,.((((((.((((((((((((.....))))))).(((((((((((.....
MSTRG.2.14,chr2L,9842,20015,AAUAAAUUCGGCCACAAUUAAUAUACUUAUUUAUUCAUCAAAACCA...,.......((((((...........(((..((((((((((((((......
MSTRG.2.16,chr2L,10953,21424,AAAUAUACCUUCUAAAAAAUAUAUGAUUUUUUCGAUAGAGAUACAC...,...................((((((........(((((((((((.....
MSTRG.2.17,chr2L,13484,20015,AAUAAAUUCGGCCACAAUUAAUAUACUUAUUUAUUCAUCAAAACCA...,.........((((..............((((((((((((((((......
...,...,...,...,...,...
MSTRG.24539.1,chrY,3637723,3640688,GUGGAAAAAUAUUCUUAGGAAUUACCAAACACUAUAAAAACAAAAU...,(((.......((((....)))).......)))...((((((........
MSTRG.24540.1,chrY,3648684,3649117,CGUCUAAUCAGUAUAUGUGUACAAAAAGAACGAAUAAAGAACGGUA...,.......((((.((((.(((.((((...............((((((...
MSTRG.24541.2,chrY,3665131,3666483,GAAGUGGAGCAGGUGUAGAAAUAGGAAGAGAACUAGGGGUAAAUAG...,...(((((((.((((((((..(((((((.(((...(((((...(((...


In [16]:
inference["proba"] = pd.DataFrame(predictor.predict_proba(inference, validate=False)[:, 1], index=inference.index)

In [17]:
inference["pred"] = predictor.predict(inference.drop(columns=["proba"]), validate=False)

In [18]:
res = res.merge(inference[['proba', 'pred']], how='right', left_index=True, right_index=True)

In [19]:
import pandas as pd
from pathlib import Path

sources = Path(base_path).glob('*/t_data.ctab')

paths = {p.parent.name: p for p in sources}

def parse_ctabs(paths: dict[str, Path]) -> pd.DataFrame:
    dfs = []
    for name, path in paths.items():
        df = pd.read_csv(path, sep='\t', usecols=['t_name', 'FPKM'])
        df = df.rename(columns={'FPKM': f'fpkm_{name.lower()}'})
        df.drop_duplicates(subset=['t_name'], inplace=True)
        dfs.append(df)
    merged_df = dfs[0]
    for df in dfs[1:]:
        merged_df = merged_df.merge(df, on='t_name', how='outer')
    return merged_df

def fpkm_stats(fpkm_df: pd.DataFrame) -> pd.DataFrame:
    fpkm_values = fpkm_df.drop(columns=['t_name'])
    fpkm_df['fpkm_mean'] = fpkm_values.mean(axis=1)
    fpkm_df['fpkm_median'] = fpkm_values.median(axis=1)
    fpkm_df['fpkm_std'] = fpkm_values.std(axis=1)
    return fpkm_df

fpkm = fpkm_stats(parse_ctabs(paths))

In [20]:
fpkm

,t_name,fpkm_srr18432800,fpkm_srr18432801,fpkm_srr18432802,fpkm_srr18432803,fpkm_srr18432804,fpkm_srr18432805,fpkm_srr18432809,fpkm_srr18432810,fpkm_srr18432811,fpkm_srr18432812,fpkm_srr18432813,fpkm_srr18432814,fpkm_mean,fpkm_median,fpkm_std
0,FBgn0013687_df_pt,0.059986,0.047586,0.000000,0.023756,0.038929,0.085387,0.000000,0.033986,0.000000,0.055654,0.000000,0.026073,0.030946,0.030030,0.028080
1,FBti0015567-RA,1.683359,1.241024,1.494745,1.683366,1.624841,2.337746,0.000000,1.774932,0.920289,1.211146,0.507317,0.815212,1.274498,1.367884,0.635393
2,FBti0018861-RA,4.445781,2.787519,2.209614,3.244210,2.841586,4.435802,3.991360,2.854652,2.556868,2.029114,1.563118,1.898288,2.904826,2.814553,0.963847
3,FBti0018862-RA,5.336230,3.416794,2.514294,3.818548,3.420164,5.232962,2.558433,3.720513,2.927633,2.447002,1.892881,2.430595,3.309671,3.172213,1.092313
4,FBti0018865-RA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66304,RR50960_transposable_element-RA,3.668359,2.762678,1.644487,2.764889,1.990396,3.413096,0.000000,3.526203,3.271206,1.780352,2.373802,2.413094,2.467380,2.587886,1.033794
66305,RR51001_transposable_element-RA,0.059425,0.766703,0.102183,0.366552,0.234558,0.033299,0.000000,0.432333,0.231182,0.138013,0.314431,0.031787,0.225872,0.184597,0.221432
66306,RR51007_transposable_element-RA,7.043255,7.349313,7.286599,6.983126,5.414223,6.882892,2.204550,6.508019,14.137106,7.305468,8.886226,9.805453,7.483853,7.164927,2.789978
66307,RR51048_transposable_element-RA,9.984214,6.659973,5.742663,6.240782,6.170960,11.754229,2.510707,11.003805,3.266365,4.169394,1.385474,2.319027,5.933966,5.956812,3.469376


In [21]:
res = res.merge(fpkm, how='left', left_index=True, right_on='t_name').drop(columns=['t_name'])

In [25]:
res.pred.value_counts()

pred
False    24026
True       193
Name: count, dtype: int64

In [23]:
res.to_excel("flync_vu_gse199164_3d_vs_7d_inference_results.xlsx", index=True)